# CANDO Tutorial

This notebook will walk you through how to generate a CANDO matrix, set up a CANDO object, probe the data, benchmark the platform, and make therapeutic predictions. 

## ToC
* [Introduction](#intro)
* [Get Started](#get-started)
* [Generate interaction matrix](#interaction-matrix)
* [Setting up CANDO objects](#cando-object)
* [Check data](#check-data)
* [CANBENCHMARK](#canbenchmark)
* [CANPREDICT](#canpredict)
* [Machine Learning with CANDO](#ml)
* [Custom protein subsets and signatures](#custom)
* [Generate signatures for novel compounds](#novel)
* [Generate a new compound library](#new-cmpd-lib)
* [Virtual screening in CANDO](#virtual-screen)



<a class="anchor" id="intro"></a>
## Introduction

<a class="anchor" id="get-started"></a>
## Get Started

We begin by importing the cando package. Once cando has been imported we can pull the example mappings and matrix data for this tutorial, `cnd.get_tutorial()`. Lastly, for our set up, we define important variables that will be used throughout this tutorial. These variables will be explained as each becomes important for different functions.

In [1]:
import sys, os
# import the cando package
import cando as cnd

# Pull data for this entire tutorial
cnd.get_tutorial()
cnd.get_data()

# Define variables for matrix generation and CANDO object creation
matrix_file='tutorial_matrix-approved.tsv'
cmpd_map='cmpds-v2.2.tsv'
ind_map='cmpds2inds-v2.2.tsv'
#prot_scores='{}/example-prots_scores.tsv'.format(root_path)
protein_set="tutorial-bac-prots.txt"
dist_metric='cosine'
ncpus=3

os.chdir("tutorial")

All data for tutorial downloaded.

/Users/williammangione/Documents/CANDO/cando/data/v2.2+/mappings/drugbank-v2.2.tsv exists.
/Users/williammangione/Documents/CANDO/cando/data/v2.2+/mappings/drugbank2ctd-v2.2.tsv exists.
/Users/williammangione/Documents/CANDO/cando/data/v2.2+/prots/nrpdb-coach.tsv exists.
All data for v2.2 downloaded.


<a class="anchor" id="interaction-matrix"></a>
## Generate interaction matrix

The example interaction matrix, `matrix_file`, has already been downloaded via `get_tutorial()`. This function may take anywhere from ~1-10 mins on 3 cores, depending on your computer's processor.

In this step we will generate a matrix of 2,449 approved compounds by 64 proteins, populated with the corresponding interaction score betwen each drug and protein. The final matrix will have drugs as the columns (indexed according to the compound mapping file), and proteins as the indices (indexed by PDB and chain ID).

The function `generate_matrix()` first creates a dataframe for all chemical fingerprint similarity scores comparing each compound in the specified version library `v` (or only approved drugs with `approved_only=True`) to every potential binding site ligand from the PDB using RDKit to compute the chemical fingerprints in which `fp` denotes the type/radius of fingerprint (rd_ecfp4, rd_ecfp8, etc). The vector type, `vect`, can be binary ("1024_bit" or "2048_bit") or integer ("int") and denote the presence or absence or count of molecular substructures in the molecule, respectively. If using integer vectors, `dist` should be set to "dice", but binary vectors can be set to Tanimoto ("tani"). Next, it creates a dataframe of all potential binding sites for each protein in the specified protein library, `protlib`, with their corresponding binding site scores from the specified binding site prediction method, `bs` ("coach", "cof", "ssite", "tms"). The function then iterates over all drugs and protein binding sites to populate the matrix with the best score based on the following input parameters: 1) `i_score` - the scoring protocol of choice, which can be 'C', 'dC', 'P', 'CxP', 'dCxP', where C is the fingerprint similarity score (either Tanimoto or Sorenson-Dice coefficient), dC is the percentile of the C score for the compound compared to all ligands in the library, and P is binding site score associated with the ligand predicted by the [COACH algorithm](https://zhanglab.ccmb.med.umich.edu/COACH/), 2) `c_cutoff` and `p_cutoff` - set cutoffs which ignore any C or P scores below each threshold, respectively, and 3) `percentile_cutoff` - similar to `c_cutoff` but with the dC score (overrides `c_cutoff` if not `None`). These scores serve as a proxy for binding strength/probability of the drug and protein target. We then output the matrix to a tsv file, `out_file`, which in this case is "tutorial_matrix-all.tsv" (`out_path` can be set to write the file to a specific directory).

We will create a matrix using the 'CxP' protocol, which just chooses the top Dice score to any ligand predicted to bind to a given protein, without any cutoffs. This function is parallelized, so setting the variable `ncpus` will change the number of processors that are used for this function. NOTE: percentile cutoff protocols ('dC', 'dCxP') take much longer to compute than 'C' and 'P' protocols. 

In [2]:
# generate example cando interaction matrix (2,449 compounds x 64 proteins)
cnd.generate_matrix(v="v2.2", fp="rd_ecfp4", vect="int",
                    dist="dice", org="tutorial", bs="coach",
                    c_cutoff=0.0, p_cutoff=0.0, percentile_cutoff=0.0,
                    i_score="CxP", out_file='tutorial_matrix-approved.tsv', out_path=".",
                    nr_ligs=True, approved_only=True,
                    lib_path='', lig_name=False, ncpus=ncpus)

Generating CANDO matrix...
Matrix written to ./tutorial_matrix-approved.tsv.
Matrix generation took 4 min 0 s to finish.


<a class="anchor" id="cando-object"></a>
## Setting up CANDO object

1. First argument, `cmpd_map`, is the compound mapping which specifies all of the compounds in the matrix by name and ID.

2. Second argument, `ind_map`, is the indication mapping which specifies which diseases the drugs/compounds are approved to treat.

3. `matrix` is the cando interaction matrix file which contains the names of the proteins and the scores to all compounds in the proper order/indices as the compound mapping file indicates. **This file must be in tsv format.**

4. `compute_distance` tells the object to compute the distances between the compounds in the platform based on the similarity of their interactions with all of the proteins. The distance metric used, `dist_metric`, can be cosine, RMSD, or many other common distance metrics (default is RMSD). This can be relatively time consuming depending on the amount of proteins, but with proteins on the order of hundreds it shouldn't take longer than a minute or two. If the computation does take a while (let's say >100,000 proteins), you can add a `save_dists='name_of_dist_file.tsv'` flag to save the computation. Then, simply use the `read_dists='name_of_dist_file.tsv'` to read in the already computed distances to save time. 

5. `compound_set` indicates which compounds to include in the CANDO object. The default option is 'all', but in this case we will use the 'approved' set to match the matrix we computed earlier. 

There are many other options available for the CANDO object instantiation, please consult the README for all of them (some more will be covered in this tutorial).

In [3]:
# create object using example cando interaction matrix
# compute distances using rmsd distance metric
cando = cnd.CANDO(cmpd_map, ind_map, matrix=matrix_file, compound_set='approved', compute_distance=True, 
                  dist_metric=dist_metric, ncpus=ncpus)

Reading signatures from matrix...
Done reading signatures.

Computing cosine distances...
Done computing cosine distances.



<a class="anchor" id="check-data"></a>
## Check data
The imported data from `get_tutorial()` contains the CANDO v2.2 mappings with a sample of 64 proteins (for simplicity sake). It should contain the v2.2 set of 13,190 drugs/compounds and 2,178 indications. Let's check to make sure this is the case. The `CANDO` object should automagically assign each compound its signature of 64 proteins. 

We can also search for compounds using a string input with `search_compound()`, in case you are unaware if the mapping file contains a compound of interest. Don't worry about exact spelling, it will output the top 5 (default) closest matches to your query. 

In [4]:
# print cando object stats
print('compounds', len(cando.compounds))
print('indications', len(cando.indications))
print('proteins', len(cando.proteins))
print('')

cando.search_compound('bilavarudin')

compounds 2449
indications 2214
proteins 64

id	name
0	bivalirudin
590	lamivudine
7272	ivabradine
1129	telbivudine
5056	nilvadipine


Spelling was a bit off, but we found bivalirudin!

Let's make sure bivalirudin has 64 values in its signature and take a look at the values themselves. We know its `id_` is 0 from searching. Depending on the scoring protocol used, the range/distribution of these values will vary.

The `compute_distance` flag above tells the CANDO object to compute the RMSD between each compound on an all-vs-all basis. Let's check out the top5 most similar compounds to bivalirudin. The `Compound.similar` lists are part of the `Compound` objects and contain a tuple of every other compound object and its computed distance.

In [5]:
# print bivalirudin signature
c = cando.compounds[0]
print(c.name, len(c.sig))
print(c.sig)
print('')

# top5 most similar compounds to bivalirudin
for s in c.similar[0:5]:
    print(s[0].name, round(s[1], 3))

bivalirudin 64
[0.437, 0.087, 0.079, 0.077, 0.341, 0.062, 0.037, 0.061, 0.053, 0.054, 0.141, 0.094, 0.077, 0.328, 0.0, 0.071, 0.591, 0.411, 0.511, 0.488, 0.121, 0.023, 0.052, 0.375, 0.526, 0.088, 0.087, 0.038, 0.273, 0.344, 0.389, 0.574, 0.109, 0.254, 0.228, 0.094, 0.487, 0.0, 0.119, 0.06, 0.146, 0.071, 0.032, 0.228, 0.33, 0.063, 0.113, 0.396, 0.05, 0.046, 0.0, 0.064, 0.389, 0.068, 0.086, 0.433, 0.238, 0.404, 0.102, 0.175, 0.178, 0.355, 0.255, 0.084]

leuprolide 0.024
goserelin 0.026
gramicidin_d 0.024
desmopressin 0.059
cetrorelix 0.022


<a class="anchor" id="canbenchmark"></a>
## Canbenchmark - benchmarking module
An important part of CANDO is benchmarking how well we can recapture drugs known to treat the same diseases within their respective `Compound.similar` lists. Currently, our benchmarking code calculates three metrics:
1. Average indication accuracy (aia) - this value is the average of every individual indication accuracy
2. Average pairwise accuracy (apa) - this value is the weighted average of each individual indication accuracy based on the number of drugs approved to treat it
3. Indication coverage (ic) - this is the count of the number of non-zero indication accuracies

The higher the metric scores for a given matrix/compound-protein scoring protocol, the more confidence we have in making predictions. 

An accuracy is calculated for each indication that has at least 2 compounds associated. To do this, we hold out each compound and look for any of the other approved compounds within a certain cutoff of the `Compound.similar` list for the held-out compound. The cutoffs are predetermined to be top10, top25, top50, and top100. There are also percent cutoffs that vary based on the number of compounds in the platform. So, for Indication-A with three drugs approved (D1, D2, and D3), we would hold out D1 and look for *either* D2 or D3 in the top10, top25, top50, and top100 compounds to D1. This would be repeated for D2 and D3. So let's say D3 was recaptured at rank 5 for D1, D1 was recaptured at rank 12 for D2, and D1 was recaptured at rank 27 for D3. The top10 average indication accuracy would be (1+0+0) / 3 == 33%, whereas the top25 would be (1+1+0) / 3 == 66%, and the top50 would be 100%. 

### Canbenchmark - classic
Below is how to run the benchmarking code. The first argument of `canbenchmark()` is the extension to put on the output files, which are "results_analysed_named", "raw_results", and "summary". 

In [6]:
cando.canbenchmark('tutorial')

	aia
top10	20.316
top25	26.808
top50	32.635
top100	41.836
top2449	100.000
top1%	26.186
top5%	44.843
top10%	57.426
top50%	89.378
top100%	100.000




Below is the printed summary file for the classic canbenchmark.

In [7]:
with open("summary-tutorial.tsv", 'r') as f:
    for line in f:
        print(line.strip('\n'))

	top10	top25	top50	top100	top2449	top1%	top5%	top10%	top50%	top100%
aia	20.316	26.808	32.635	41.836	100.000	26.186	44.843	57.426	89.378	100.000
apa	35.180	47.112	57.909	70.206	100.000	46.230	73.574	83.817	97.693	100.000
ic	819	907	969	1078	1595	904	1112	1254	1532	1595



### Canbenchmark - associated
There are variations of the benchmarking code which may be of interest to some users. For example, not every compound in the mapping files are associated with a disease. This can add significant noise and decrease performance if the compound library is very large. We can have these compounds removed by running another benchmarking method called `canbenchmark_associated()`, which will automatically filter out these non-associated compounds. 

In [8]:
cando.canbenchmark_associated('tutorial_associated')

Making CANDO copy with only benchmarking-associated compounds
Reading signatures from matrix...
Done reading signatures.

Computing cosine distances...
Done computing cosine distances.

	aia
top10	22.247
top25	30.538
top50	38.878
top100	49.696
top1464	100.000
top1%	24.593
top5%	44.549
top10%	56.129
top50%	87.369
top100%	100.000




Below is the printed summary file for the associated canbenchmark.

In [9]:
with open("summary-tutorial_associated.tsv", 'r') as f:
    for line in f:
        print(line.strip('\n'))

	top10	top25	top50	top100	top1464	top1%	top5%	top10%	top50%	top100%
aia	22.247	30.538	38.878	49.696	100.000	24.593	44.549	56.129	87.369	100.000
apa	40.433	54.741	66.853	78.057	100.000	45.016	73.141	82.941	97.165	100.000
ic	827	946	1058	1176	1595	862	1123	1249	1526	1595



### Canbenchmark - continuous
Yet another variation of our benchmarking is `continuous=True`. This method identifies percentiles for the compound distances which define our cutoffs. Rather than the top10 compounds, we can define a distance cutoff that will not punish a compound that may fall at rank 11, but is still very similar to the hold-out compound (think of large "me-too" families of compounds). These distance cutoffs are determined empirically after the distances are calculated/read. 

In [10]:
cando.canbenchmark('tutorial_continuous', continuous=True)

	aia
0.1%ile	9.679
.25%ile	12.527
0.5%ile	16.498
1%ile	21.796
5%ile	36.013
10%ile	46.204
20%ile	59.283
33%ile	72.732
50%ile	83.878
100%ile	100.000




Below is the printed summary file for the continuous canbenchmark.

In [11]:
with open('summary-tutorial_continuous.tsv', 'r') as f:
    for line in f:
        print(line.strip('\n'))

	0.1%ile	.25%ile	0.5%ile	1%ile	5%ile	10%ile	20%ile	33%ile	50%ile	100%ile
aia	9.679	12.527	16.498	21.796	36.013	46.204	59.283	72.732	83.878	100.000
apa	15.771	21.199	28.432	36.552	56.932	66.668	77.154	85.143	91.447	100.000
ic	546	604	681	768	949	1074	1226	1363	1460	1595



<a class="anchor" id="canpredict"></a>
## Canpredict - drug-disease association prediction module

An important part of CANDO is generating putative drug candidates for a specific disease and predicting indications for which added or current drugs in our library can be therapetuic. We can do this with the canpredict functions.

### Canpredict - compounds
Generating putative drug candidates for a specific disease is one way we may want to use the predictive power of our platform. For this, we use the `canpredict_compounds()` function, which basically uses a consensus method to rank putative candidates based on how many times they show up as similar to drugs that are associated to the disease, within some defined cutoff. The default cutoff is 10 (the most stringent from benchmarking), but this can be varied. In its current implementation, `canpredict_compounds()` ranks compounds based on the consensus count and uses the average rank of those as the tie-breaker. 

Let's make predictions for human immunodeficiency virus (HIV). First, let's search our indication database for the appropriate MeSH ID. 

In [12]:
cando.search_indication('HIV')

Matches exactly containing HIV:
id             	name
MESH:D015658	HIV Infections
MESH:D006679	HIV Seropositivity
MESH:D019247	HIV Wasting Syndrome
MESH:D039682	HIV-Associated Lipodystrophy Syndrome

Matches using string distance:
id             	name
MESH:D015658	HIV Infections


The one we want is "HIV Infections" with an `id_` of "MESH:D015658". This is the input for `canpredict_compounds()`, but first let's see more about this indication. 

In [13]:
hiv = cando.get_indication("MESH:D015658")
print('Number of drugs associated with {}: {}'.format(hiv.name, len(hiv.compounds)))
for cmpd in hiv.compounds:
    print('{}\t{}'.format(cmpd.id_, cmpd.name))

Number of drugs associated with HIV Infections: 32
509	efavirenz
1098	saquinavir
5094	beta_carotene
384	zidovudine
1151	fosamprenavir
918	abacavir
590	lamivudine
599	adefovir_dipivoxil
818	zalcitabine
587	delavirdine
115	nelfinavir
826	isoniazid
1399	deferasirox
225	methadone
4402	maraviroc
877	hydroxyurea
584	amprenavir
940	atazanavir
776	didanosine
392	ritonavir
5142	raltegravir
4903	etravirine
308	miglustat
119	indinavir
132	nevirapine
58	vitamin_a
413	foscarnet
689	ribavirin
1391	lopinavir
911	thalidomide
533	stavudine
618	pentamidine


There are 32 drugs associated with HIV Infections, which makes sense given it is well-studied. Most seem to be protease inhibitors (saquinavir, fosamprenavir) or nucleoside analogs (zalcitabine, didanosine). 

We can now make predictions for other drugs/compounds that may be useful against HIV. This list is very long due to the large amount of drugs approved to treat HIV, so we should look at only the top10 (default) by setting `n=10`. We will also only print the first 10 compounds (`topX=10`).

In [14]:
cando.canpredict_compounds("MESH:D015658", n=10, topX=10)

32 compounds found for MESH:D015658 --> HIV Infections
Generating compound predictions using top10 most similar compounds...

rank	score1	score2	id	approved	name
1	7	4.3	1128	true    	darunavir
2	4	3.0	756	true    	emtricitabine
3	4	5.0	331	true    	entecavir
4	4	5.2	2982	true    	brivudine
5	4	5.8	7854	true    	lifitegrast
6	4	5.8	321	true    	trifluridine
7	4	6.2	574	true    	moexipril
8	4	7.0	4817	true    	regadenoson
9	4	7.0	143	true    	idoxuridine
10	3	2.0	7256	true    	cobicistat




Perhaps using the top10 compounds is not encompassing enough. Let's change it to top25 (n=25) and see if the predictions change. We will still print the first 10 compounds.

In [15]:
cando.canpredict_compounds("MESH:D015658", n=25, topX=10)

32 compounds found for MESH:D015658 --> HIV Infections
Generating compound predictions using top25 most similar compounds...

rank	score1	score2	id	approved	name
1	8	5.0	1128	true    	darunavir
2	7	11.3	1095	true    	paclitaxel
3	7	12.7	1126	true    	decitabine
4	6	6.0	756	true    	emtricitabine
5	6	8.5	7854	true    	lifitegrast
6	6	9.3	574	true    	moexipril
7	6	10.2	136	true    	cladribine
8	6	11.0	9053	true    	macimorelin
9	6	12.3	515	true    	clofarabine
10	6	13.2	5106	true    	cabazitaxel




We can show more results by changing the `topX` parameter. Let's print 25. 

We can also show the compounds that are already approved for 'HIV Infections' - indicated by an asterisk in the 'approved' column, by changing the `keep_associated` parameter to `True`. 

<span style="color:red">(Note: this function is heavily reliant on the input indication mapping.)</span>

In [16]:
cando.canpredict_compounds("MESH:D015658", n=25, topX=25, 
                           keep_associated=True)

32 compounds found for MESH:D015658 --> HIV Infections
Generating compound predictions using top25 most similar compounds...

rank	score1	score2	id	approved	name
1	8	5.0	1128	true    	darunavir
2	7	5.1	584	true*   	amprenavir
3	7	10.0	1098	true*   	saquinavir
4	7	11.3	1095	true    	paclitaxel
5	7	12.7	1126	true    	decitabine
6	6	6.0	756	true    	emtricitabine
7	6	8.5	7854	true    	lifitegrast
8	6	9.3	574	true    	moexipril
9	6	10.2	136	true    	cladribine
10	6	11.0	9053	true    	macimorelin
11	6	12.3	515	true    	clofarabine
12	6	13.2	5106	true    	cabazitaxel
13	6	14.0	1151	true*   	fosamprenavir
14	6	14.2	7855	true    	velpatasvir
15	6	17.7	776	true*   	didanosine
16	5	5.8	119	true*   	indinavir
17	5	7.8	331	true    	entecavir
18	5	12.4	7431	true    	eluxadoline
19	5	14.2	969	true    	capecitabine
20	5	14.2	918	true*   	abacavir
21	5	17.0	10070	true    	ivosidenib
22	5	19.4	198	true    	mitomycin
23	4	5.2	2982	true    	brivudine
24	4	5.8	321	true    	trifluridine
25	4	5.8	115	true* 

In this case, we recaptured 7 associated therapies for HIV, including amprenavir, saquinavir, fosamprenavir, didanosine, indinavir, abacavir, and nelfinavir.

### Canpredict - de novo

Sometimes there are no drugs associated with an indication (whether it be in reality or in your input indication mapping); in those cases we can use the `canpredict_denovo()` function to suggest putative candidates. Basically, this function counts/sums the number of protein interaction scores above a set threshold for each compound and ranks them according to frequency and strength. This is particularly useful for pathogen proteomes (like SARS-CoV-2 or bacterial proteomes) or for finding which compounds target a subset of proteins of interest (e.g. kinases). In our case, we have a diverse set of proteins, which would render the results meaningless. Instead, we can input a list of protein IDs in which we are interested. We have precompiled a list of bacterial proteins already present in the sample matrix for convenience. Note: if we had read in an indication-genes mapping file, we could use the `ind_id=` parameter to automatically select all proteins associated with said indication. 

In [17]:
bacterial_proteins = ["3mk7C", "3eziA", "1u2mC", "3atsA", "2x4mA", 
                      "4wliA", "1t4aA", "4zxkA", "1zhhA", "1eb0A", 
                      "4nqwB", "2gqrA", "2qlcA", "3rf1A", "2xpwA"]

cando.canpredict_denovo(method='count', threshold=0.6, topX=30, proteins=bacterial_proteins)

Printing the 30 highest predicted compounds...

rank	score1	score2	id	approved	name
1	2	1.5	10026	true    	magnesium
2	2	1.38	1384	true    	zinc
3	1	0.9	41	true    	glycine
4	1	0.85	24	true    	aspartic_acid
5	1	0.79	27	true    	adenosine_phosphate
6	1	0.77	639	true    	tetracycline
7	1	0.76	7602	true    	potassium_hydroxide
8	1	0.76	5059	true    	potassium_iodide
9	1	0.76	1171	true    	potassium_cation
10	1	0.76	641	true    	potassium_chloride
11	1	0.7	889	true    	minocycline
12	1	0.69	7306	true    	copper
13	1	0.67	1191	true    	calcium
14	1	0.667	524	true    	adenosine
15	1	0.667	89	true    	vidarabine
16	1	0.661	479	true    	oxytetracycline
17	1	0.657	69	true    	asparagine
18	1	0.653	53	true    	nadh
19	1	0.651	1149	true    	rolitetracycline
20	1	0.651	38	true    	glutamic_acid
21	1	0.63	3781	true    	fructose
22	1	0.622	8204	true    	sarecycline
23	1	0.619	150	true    	lymecycline
24	1	0.616	7282	true    	chlortetracycline
25	1	0.615	57	true    	valine
26	1	0.612	3936	true    	c

Interestingly, multiple antibiotics receive scores higher than the cutoff of 0.6, including **tetracycline, minocycline, oxytetracycline, sarecycline**, among others. 

### Canpredict - indications


Below we print the first 10 indications predicted for Paromomycin (CANDO ID 1225) using the top10 most similar compounds. Again, this tallies how many times specific diseases show up as associated with the top10 most similar compounds to paromomycin. 

In [18]:
paro = cando.get_compound(1225)
print(paro.name)
cando.canpredict_indications(paro, 
                             n=10, topX=10)

paromomycin
Using CANDO compound paromomycin
Compound has id 1225 and index 1225
Comparing signature to all CANDO compound signatures...
Generating indication predictions using top10 most similar compounds...
Printing the 10 highest predicted indications...

rank	score	ind_id    	indication
1	4	MESH:D004927	Escherichia coli Infections
2	4	MESH:D008581	Meningitis
3	3	MESH:D004697	Endocarditis, Bacterial
4	3	MESH:D007710	Klebsiella Infections
5	3	MESH:D011014	Pneumonia
6	3	MESH:D014376	Tuberculosis
7	3	MESH:D014552	Urinary Tract Infections
8	3	MESH:D011512	Proteus Infections
9	2	MESH:D000380	Agranulocytosis
10	2	MESH:D016470	Bacteremia



Not surprisingly, all of the predictions are infections of some sort (aka: most antibiotics share a great deal of structural similarity with each other). 

### Similar compounds
`similar_compounds()` prints the first `n` most similar compounds for a given compound. This, like `canpredict_indications()` can be used with cando compounds, `cando_cmpd`, or novel compounds with a signature file (we will explore this later).

Below we print the first 10 most similar compounds to Paromomycin.

In [19]:
paro = cando.get_compound(1225)
cando.similar_compounds(paro, n=10)

Using CANDO compound paromomycin
Compound has id 1225 and index 1225
Comparing signature to all CANDO compound signatures...
Printing top10 most similar compounds...

rank	dist	id	name
1	0.001	341	framycetin
2	0.006	368	amikacin
3	0.008	3258	ribostamycin
4	0.008	1039	kanamycin
5	0.010	178	acarbose
6	0.014	950	streptomycin
7	0.016	7063	hyaluronic_acid
8	0.018	8533	steviolbioside
9	0.019	567	tobramycin
10	0.019	9205	dibekacin
11	0.023	465	lactulose




Unsurprisingly, the most similar compounds to paromomycin are known antibiotics.  

<a class="anchor" id="ml"></a>
## Machine learning with CANDO
The "proteomic vectors" within CANDO lend themselves well to machine learning to perhaps learn more complex relationships between the proteins within the vector and their impacts on the treatment of diseases. CANDO has built-in ML algorithms that allow for two main functionalities:
1. Benchmark the platform using a hold-one-out protocol very similar to canbenchmark
2. Make predictions for novel or non-associated compounds that may be therapeutic for a given disease

The ML module currently supports 2 algorithms: random forests and logistic regression. The models are trained on drugs approved for the disease (positive classes) and an equal number of randomly selected "neutral samples", which are drugs/compounds not approved for the disease (negative samples). Random seeds may be set to ensure the same compounds are used in training. 

### ML - benchmark
We have the option to benchmark the platform with an ML algorithm - this module outputs files very similar to canbenchmark. For this tutorial, we will skip this function as it requires a great deal of time to complete (training a separate model for EVERY drug-disease association, basically). The command to do so with a logistic regression classifier is below, feel free to run it! The `'out='` flag defines the name of the output files. Again, only diseases with 2+ compounds associated are benchmarked. 

`cnd.ml(method='rf', benchmark=True, seed=50, out='test_rf')`

### ML - predict
We can also use this module to predict if a certain compound may be therapeutic for a given disease. We can use the 
`'predict='` flag to specify a list of compounds that we wish to predict with the classifier. Let's use three drugs, imatinib, buprenorphine, and lisdexamfetamine, and see if they are predicted to have antibiotic activity using a random forest classifier. 

In [20]:
baci = cando.get_indication('MESH:D001424')

imat = cando.get_compound(503)
bup = cando.get_compound(797)
lamf = cando.get_compound(1121)

cando.ml(method='rf', effect=baci, benchmark=False, seed=50, predict=[imat, bup, lamf])

Indication: Bacterial Infections
Leave-one-out cross validation: TP=31, FP=55, FN=24, TN=0, Acc=28.182
	Compound	Prob
	imatinib	0.440
	buprenorphine	0.370
	lisdexamfetamine	0.340


The accuracy is quite poor, so fine-tuning and hyperparameter optimization is necessary to enhance performance. 

<a class="anchor" id="custom"></a>
## Custom protein subsets and signatures
It may be useful for some users to probe compound-protein interaction similarity, but only in the context of a few particular proteins (e.g. set of kinase inhibitors). Instead of generating a new matrix with all of these proteins and their corresponding interaction values, which can begin to take up a lot of storage if done multiple times, the `'protein_set='` flag can be specified during the instantiation of the CANDO object. This flag contains the path to the protein subset the user wishes to use, which is simply a list of UniProt protein IDs. The CANDO object will automatically check for each ID if it either simply matches any UniProt IDs within the matrix or if that UniProt ID is associated with any PDB chains within the matrix (based on a mapping from the SIFTs project). If there are matches, the CANDO object will now contain Compound objects with only those protein interaction values in their signatures. Below is how to use this functionality - this time we can search for the bacterial proteins from the previous `canpredict_denovo()` section, but filter the proteins from the start using a list of UniProt IDs. 

In [21]:
cando_subset = cnd.CANDO(cmpd_map, ind_map, matrix=matrix_file, compound_set='approved',
                         compute_distance=True, protein_set=protein_set,
                         dist_metric=dist_metric, ncpus=ncpus)

print("Number of proteins in new signature =", len(cando_subset.proteins))

Reading signatures from matrix...
Editing signatures according to proteins in tutorial-bac-prots.txt...
15 proteins in tutorial-bac-prots.txt mapped to 15 proteins in tutorial_matrix-approved.tsv.
Done reading signatures.

Computing cosine distances...
Done computing cosine distances.

Number of proteins in new signature = 15


The signature was successfully edited to 15 proteins. Note: this does not nececessarily mean the each UniProt ID had a corresponding PDB match -- multiple PDB chains can be associated with a given UniProt ID. 

We can also repeat all benchmarks and predictive algorithms with the new signatures. Below is the default benchmarking results with the new signatures. 

In [22]:
cando_subset.canbenchmark('test_subset')

	aia
top10	16.904
top25	24.029
top50	31.092
top100	40.775
top2449	100.000
top1%	23.625
top5%	43.785
top10%	56.529
top50%	90.123
top100%	100.000




Let's repeat the ML code from above, but this time with the reduced protein subset. 

In [23]:
baci = cando_subset.get_indication('MESH:D001424')

imat = cando_subset.get_compound(503)
bup = cando_subset.get_compound(797)
lamf = cando_subset.get_compound(1121)

cando_subset.ml(method='rf', effect=baci, benchmark=False, 
                seed=50, predict=[imat, bup, lamf])

Indication: Bacterial Infections
Leave-one-out cross validation: TP=33, FP=55, FN=22, TN=0, Acc=30.000
	Compound	Prob
	imatinib	0.410
	buprenorphine	0.320
	lisdexamfetamine	0.600


As you can see, the output probability significantly change for one of the compounds, lisdexamfetamine, from 0.340 to 0.600, illustrating the impact of protein subset composition on model behavior. 

<a class="anchor" id="novel-cmpd"></a>
## Generate proteomic signatures for new compounds 

The CANDO platform contains an extensive library of approved drugs and other compounds from DrugBank. However, if you wish to predict indications or similar drugs for a compound that is not present in our library, we make it possible with the `generate_signature()` function.

First, you must have the compound properly formatted in mol file format. There are many programs that provide conversion between many chemical file formats, such as OpenBabel.

Next, run `generate_signature()`. This will populate a tsv file with Tanimoto/Sorenson-Dice similarity scores of the provided compound to all binding site ligands in our database. These values will be used for the generation of the drug-proteome signature. The input parameters for this function are very similar to those for `generate_matrix()`, however the first argument is the path to the compound structure file in mol format. The output signature file will be saved in tsv format with the name you provide for "out_file" (with the appended path from "out_path").

**NOTE: the interaction score protocol must match the input matrix protocol, which was 'CxP' as above. If we used a different protocol, these scores would not be directly comparable.**

In [24]:
cmpd_file = "lmk235.mol"
signature_file = "lmk235_signature.tsv"

cnd.generate_signature(cmpd_file, fp="rd_ecfp4", vect="int", dist="dice", 
                      org="tutorial", bs="coach", c_cutoff=0.0, p_cutoff=0.0, 
                      percentile_cutoff=0.0, i_score="CxP", out_file=signature_file, 
                      out_path=".", nr_ligs=True)

Generating CANDO signature...
Signature written to ./lmk235_signature.tsv.
signature generation took 3 s to finish.


We then must add the compound to the existing `CANDO` object with the `add_cmpd()` function. The inputs are the newly generated signature file and the desired name (which below is "lmk-235"). 

In [25]:
cando.add_cmpd(signature_file, new_name='lmk-235')

New compound is lmk-235
New compound has id 10674 and index 10674


Now that our new compound is added to the platform, we can see what other compounds to which it is similar. We can use the `similar_compounds()` function from before to print those compounds. 

In [26]:
lmk235 = cando.get_compound(10674)
cando.similar_compounds(lmk235, n=10)

Using CANDO compound lmk-235
Compound has id 10674 and index 10674
Comparing signature to all CANDO compound signatures...
Printing top10 most similar compounds...

rank	dist	id	name
1	0.024	7830	padimate_o
2	0.025	9100	lusutrombopag
3	0.025	7659	diethylamino_hydroxybenzoyl_hexyl_benzoate
4	0.025	5103	bendamustine
5	0.025	1060	acebutolol
6	0.025	655	tirofiban
7	0.026	813	salmeterol
8	0.026	5107	capsaicin
9	0.026	7299	zucapsaicin
10	0.026	411	cinchocaine
11	0.026	7537	octinoxate




Finally, we can predict potential indications for which our new compound may be useful. Since cyclosporine is a known immunosuppressant, we should expect to find autoimmune or inflammatory indications. We can use the `canpredict_indications()` as before to print those results. 

In [44]:
cando.canpredict_indications(lmk235, n=25, topX=15)

Using CANDO compound lmk-235
Compound has id 10674 and index 10674
Comparing signature to all CANDO compound signatures...
Generating indication predictions using top25 most similar compounds...
Printing the 15 highest predicted indications...

rank	score	ind_id    	indication
1	3	MESH:D001943	Breast Neoplasms
2	3	MESH:D000787	Angina Pectoris
3	3	MESH:D001145	Arrhythmias, Cardiac
4	3	MESH:D006973	Hypertension
5	3	MESH:D004487	Edema
6	2	MESH:D001008	Anxiety Disorders
7	2	MESH:D017202	Myocardial Ischemia
8	2	MESH:D013610	Tachycardia
9	2	MESH:D014693	Ventricular Fibrillation
10	2	MESH:D018879	Ventricular Premature Complexes
11	2	MESH:D007249	Inflammation
12	2	MESH:D008113	Liver Neoplasms
13	2	MESH:D008175	Lung Neoplasms
14	2	MESH:D009374	Neoplasms, Experimental
15	2	MESH:D010146	Pain



Interestingly, both Hypertension (MESH:D006973) and Pain (MESH:D010146) are top predictions - the analgesic and hypotensive properties of lmk-235 are both supported by in vivo studies in the literature. 

<a class="anchor" id="new-cmpd-lib"></a>
## Generate a new compound library

In this part of the tutorial, we will walk you through how to create a new, unique compound library to use for matrix generation.

### Create compound set

In order to create a new compound set, you must first have a TSV (tab separated values) file that contains one of two chemical file types:

1. SMILES - `file_type='smi'`. The file must have the SMILES string as the first column and the corresponding compound name in the second column, e.g.
    `C1CNCCN(C1)S(=O)(=O)C2=CC=CC3=C2C=CN=C3 fasudil`
    
2. Mol - `file_type='mol'`. The file must have the name of the file, without the file extension. In addition, the path to the files must be given in the argument `cmpd_dir`.

In our example, we will create a new compound library of select tyrosine kinase inhibitors (TKIs) that we will then use to create a new matrix with just these drugs. First, we generate the library.

In [ ]:
cnd.add_cmpds("tki_set-test.smi", file_type='smi', fp="rd_ecfp4", vect="int", cmpd_dir=".", v='tki')

We have now generated a new compound library, located at `./tki/cmpds`. This location is partially hardcoded, so it will also create a directory in your current working directory, `./`, named after the `v` you choose, `./tki`. This makes it easier for the end-user.

Notice we chose to use the rdkit fingerprint ecfp4, `fp="rd_ecfp4"`, and the vector type int, `vect="int"`. We need to keep track of this for our matrix generation.

### Generate matrix from new compound library

Next we will generate a matrix using this **tki** compound library. The generate matrix function can take in a customized matrix, as opposed to the pregenerated/downlaoded matrices based upon our predefined `v`, e.g. v2.2, v2.3, etc. By setting `v` to the same name you used in the previous `add_cmpds` function, you can load those cmpds to create a new matrix.

We will set `v='tki'` and `lib_path='.'`. This means we will use the **tki** library, which is located in the current working dir. In addition, sicne we created our compound library using fingerprint type ecfp4, we need to make sure we use it here, as well. The remaining arguemnts has already been discussed in the previous <a class="anchor" id="interaction-matrix">generate matrix</a> section.


In [ ]:
cnd.generate_matrix(v="tki", lib_path='.',
                    fp="rd_ecfp4", vect="int",
                    dist="dice", org="tutorial", bs="coach",
                    c_cutoff=0.0, p_cutoff=0.0, percentile_cutoff=0.0,
                    i_score="CxP", out_file='', out_path="",
                    nr_ligs=True, approved_only=False,
                    lig_name=False, ncpus=ncpus)

We now have a brand new matrix with all of our new TKIs scored against our set of 64 test proteins. This file is located at `./tki/matrices/`. Again, these paths are mostly hardcoded. You can control where the matrix is written and what the name is using the `out_path` and `out_file` arguments. When these arugments are set to '' the hardcoded paths are used, as shown in this example.

<a class="anchor" id="virtual-screen"></a>
## Virtual screening in CANDO
Though the CANDO platform is mainly intended for multitarget drug discovery and repurposing, there still exists the option to check the top compound hits for a given protein. This is accomplished via the `virtual_screen()` function. Let's test the top hits for two of the known bacterial proteins from above, namely "1u2mC", "3atsA". 

In [45]:
cando.virtual_screen("1u2mC")
cando.virtual_screen("3atsA")

Protein is 1u2mC
rank	score	id	approved	name
1	0.444	317	true    	streptozocin
2	0.406	7539	true    	thiosulfuric_acid
3	0.389	1145	true    	glucosamine
4	0.35	10026	true    	magnesium
5	0.343	7300	true    	aurothioglucose
6	0.33	1191	true    	calcium
7	0.327	4140	true    	gluconolactone
8	0.306	27	true    	adenosine_phosphate
9	0.301	7291	true    	coenzyme_m
10	0.301	1715	true    	taurine

Protein is 3atsA
rank	score	id	approved	name
1	0.8	10026	true    	magnesium
2	0.67	1191	true    	calcium
3	0.58	53	true    	nadh
4	0.57	27	true    	adenosine_phosphate
5	0.53	1039	true    	kanamycin
6	0.52	524	true    	adenosine
7	0.52	89	true    	vidarabine
8	0.519	2823	true    	flavin_adenine_dinucleotide
9	0.506	1421	true    	5-o-phosphono-alpha-d-ribofuranosyl_diphosphate
10	0.473	9117	true    	inosine_pranobex



Streptozocin is the top hit for 1u2mC, which suggests it shares significant structural similarity to a ligand known/predicted to bind to this protein. Similarly, kanamycin is the 5th hit for 3atsA. These scores/ranks can change significantly based on scoring protocol used for the matrix - changing i_score in `generate_matrix()` to "dC", for example, would greatly affect the output. 